In [1]:
import numpy as np
import pyopencl as cl
import numpy.linalg as la

In [2]:
vector_dimension = 100

In [3]:
vector_a = np.random.randint(vector_dimension,size=vector_dimension) 
vector_b = np.random.randint(vector_dimension,size=vector_dimension)

In [4]:
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
context = cl.Context([device])
queue = cl.CommandQueue(context)

In [5]:
mf = cl.mem_flags 
a_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_a) 
b_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_b) 

In [6]:
program = cl.Program(context, """ 
__kernel void vectorSum(__global const int *a_g, __global const int *b_g, __global int *res_g) { 
  int gid = get_global_id(0); 
  res_g[gid] = a_g[gid] + b_g[gid]; 
} 
""").build()

In [7]:
res_g = cl.Buffer(context, mf.WRITE_ONLY, vector_a.nbytes)

In [8]:
program.vectorSum(queue, vector_a.shape, None, a_g, b_g, res_g)

In [9]:
res_np = np.empty_like(vector_a)

In [10]:
cl.enqueue_copy(queue, res_np, res_g)

In [11]:
print ("PyOPENCL SUM OF TWO VECTORS")
print ("Platform Selected = %s" %platform.name )
print ("Device Selected = %s" %device.name)
print ("VECTOR LENGTH = %s" %vector_dimension)
print ("INPUT VECTOR A")
print (vector_a)
print ("INPUT VECTOR B")
print (vector_b)
print ("OUTPUT VECTOR RESULT A + B ")
print (res_np)

PyOPENCL SUM OF TWO VECTORS
Platform Selected = Intel(R) OpenCL HD Graphics
Device Selected = Intel(R) UHD Graphics
VECTOR LENGTH = 100
INPUT VECTOR A
[10 42 13 25 31  3 19 97 52 11 86 58 97 21 25  9 84 44 67 74 46 73 75 99
 68  8  0 17 92 54 43 23 78 42 64 72 40 97 30 40 46  0 84 60 51 37 67 35
 25 67 88 84 27 52 52 76 71 17 11 19 79 43 39 18 31 56 47 39 85 14 35 35
 70 98 74 47 12 71 20  0  5 93 87 71 33 46 11 61 63 85 79 67  9 37 63 16
 48 77 13 91]
INPUT VECTOR B
[79 48  5 52 17 25 36 25 28 85 30 56  5 45 19 72 70 28 40 97 45 51 64 42
 64  1 23 14 39  1 79 88 44 39 10 44 46 92  8 63 71 58 58 24 39 34  7 74
 62 37 62 14 66 12 49 68 94  4 91 31 25 76 41 63 69  5 12 59 25 27 87 95
 92 50 13 67 12  7 51 80 25 45 18 36 72  2 91 80 43 98 55 17  2 91 62 67
 22 87 89 39]
OUTPUT VECTOR RESULT A + B 
[ 89  90  18  77  48  28  55 122  80  96 116 114 102  66  44  81 154  72
 107 171  91 124 139 141 132   9  23  31 131  55 122 111 122  81  74 116
  86 189  38 103 117  58 142  84  90  71  74 109

In [12]:
assert(la.norm(res_np - (vector_a + vector_b))) < 1e-5